In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf




class EmojiTextClassifier:
    def __init__(self) -> None:
        pass


    def load_dataset(self, dataset_path):
        df = pd.read_csv(dataset_path)
        X = np.array(df["sentence"])
        Y = np.array(df["label"], dtype = np.float64)
        return X, Y
    
    def load_feature_vector(self,vector_file):
        self.word_vectors = {}

        for line in vector_file:
            line = line.strip().split(" ")
            word = line[0]
            vector = np.array(line[1:], dtype = np.float64)
            self.word_vectors[word] = vector

    def sentence_to_feature_vectors_avg(self, sentence):
        
        try:
            sentence = sentence.lower()
            words = sentence.strip().split(" ")
            sum_vectors = np.zeros((50, ))
            for word in words:
                sum_vectors += self.word_vectors[word]

            self.avg_vector = sum_vectors / len(words)

            return self.avg_vector
        except:
            print(sentence) 



    def load_model(self):
        self.model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(5, 
                                  input_shape = (50,),
                                  activation = "softmax")
        ])

        self.model.compile(
            tf.keras.optimizers.Adam(),
            loss = "categorical_crossentropy",
            metrics = ["accuracy"]
        )


    def train(self, X_train, Y_train):
        self.model.fit(X_train, Y_train, epochs = 250)

    def test(self, X_test, Y_test):
        self.model.evaluate(X_test, Y_test)    

    def label_to_emoji(self, label):
        emojies = ["❤️","⚽️","😄","😔","🍽"]
        return emojies[label]
    

    def inference(self, test_sentence):
        test_sentence  = np.array([test_sentence])
        result = self.model.predict(test_sentence)
        y_pred = np.argmax(result)
        label = self.label_to_emoji(y_pred)

        return label

        



if __name__ == "__main__":
    
    train_dataset = "dataset/train.csv"
    test_dataset= "dataset/test.csv"
    vector_file = open("glov.6B/glove.6B.50d.txt", encoding = "utf-8")

    X_train_avg = []
    X_test_avg = []

    textclassifier = EmojiTextClassifier()
    X_train, Y_train = textclassifier.load_dataset(train_dataset)
    X_test, Y_test = textclassifier.load_dataset(test_dataset)

    textclassifier.load_feature_vector(vector_file)

    for x_train in X_train:
        X_train_avg.append(textclassifier.sentence_to_feature_vectors_avg(x_train))
        
    X_train_avg = np.array(X_train_avg)


    for x_test in X_test:
        X_test_avg.append(textclassifier.sentence_to_feature_vectors_avg(x_test))

    X_test_avg = np.array(X_test_avg)

    
    Y_train_one_hot = tf.keras.utils.to_categorical(Y_train, num_classes = 5)
    Y_test_one_hot = tf.keras.utils.to_categorical(Y_test, num_classes = 5)

    textclassifier.load_model()

    textclassifier.train(X_train_avg, Y_train_one_hot)
    textclassifier.test(X_test_avg, Y_test_one_hot)


    mysentense = textclassifier.sentence_to_feature_vectors_avg("I like pizza")
    result = textclassifier.inference(mysentense)
    print(result)





2024-01-19 23:56:24.327918: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-19 23:56:24.396424: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 23:56:24.396491: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 23:56:24.398952: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 23:56:24.410064: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-19 23:56:25.854065: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

i will  run


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (56,) + inhomogeneous part.